In [2]:
import requests
from bs4 import BeautifulSoup
# import pandas as pd
from urllib.parse import urljoin
import json

In [6]:
base_url="https://www.joghr.org/article/94931-understanding-key-factors-for-strengthening-nepal-s-healthcare-needs-health-systems-perspectives"
visited_urls=set()
urls_to_visit=[base_url]
max_documents=20
number_of_documents=0
scraped_data=[]

In [7]:
while urls_to_visit and len(visited_urls) < max_documents:
    current_url=urls_to_visit.pop()
    if current_url in visited_urls:
        continue
    
    try:
        print(f"{current_url}")
        response=requests.get(current_url,timeout=10)
        print(response.status_code)
        if response.status_code!=200:
            continue

        # print(response.text)
        print(type(response.text))
        soup=BeautifulSoup(response.text,"html.parser")
        print(type(soup))
        # print(soup)
        
        title=soup.title.text
        paragraphs=soup.find_all('p')
        # print(paragraphs)
        content="".join([p.text for  p in paragraphs])
        # print(content)
        # print(title)
        scraped_data.append({'url':current_url,'title':title,'content':content})
        visited_urls.add(current_url)
        number_of_documents+=1
        
        for link in soup.find_all('a',href=True):
            abosulteUrl=urljoin(current_url,link['href'])
            if abosulteUrl not in visited_urls and abosulteUrl.startswith("http"):
                urls_to_visit.append(abosulteUrl)
                
    except Exception as e:
        print(f"Error crawling {current_url}: {e}")

    

https://www.joghr.org/article/94931-understanding-key-factors-for-strengthening-nepal-s-healthcare-needs-health-systems-perspectives
200
<class 'str'>
<class 'bs4.BeautifulSoup'>
https://scholasticahq.com/
200
<class 'str'>
<class 'bs4.BeautifulSoup'>
https://www.facebook.com/scholasticahq
200
<class 'str'>
<class 'bs4.BeautifulSoup'>
https://twitter.com/scholasticahq
200
<class 'str'>
<class 'bs4.BeautifulSoup'>
https://scholasticahq.us2.list-manage.com/track/click?u=a31d40b633c1d0c4f04737046&id=511fc0f24e&e=84a6fa8f43
999
https://scholasticahq.com/predatory
200
<class 'str'>
<class 'bs4.BeautifulSoup'>
https://scholasticahq.us2.list-manage.com/track/click?u=a31d40b633c1d0c4f04737046&id=511fc0f24e&e=84a6fa8f43
999
https://scholasticahq.com/privacy-policy
200
<class 'str'>
<class 'bs4.BeautifulSoup'>
https://scholasticahq.us2.list-manage.com/track/click?u=a31d40b633c1d0c4f04737046&id=511fc0f24e&e=84a6fa8f43
999
https://scholasticahq.com/law-review-submission-season-hq
200
<class 'str'>

In [ ]:
print(number_of_documents)
if scraped_data:
    with open('healthcaredata.json', 'w', encoding='utf-8') as f:
        json.dump(scraped_data, f, ensure_ascii=False, indent=4)
    print("Data saved to healthcaredata.json")
else:
    print("No data was scraped.")